In [ ]:
import numpy as np
import pandas as pd
import pyspark as ps
import matplotlib.pyplot as plt
import csv

import unicodedata

from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from gensim import corpora, models

import pyLDAvis.gensim

In [ ]:
df = pd.read_json('reduced_tweets.json', lines=True)

## Necessary data processing:

Different languages have different pattern but no difference betweet mental illness related tweets and unrelated tweets can be found

In [ ]:
#this might be a dumb idea in retrospect. we need to keep èä to use stop word removal effectivley
# df['main'] = df['main'].astype(str).str.lower().\
#                     apply(lambda tweet: unicodedata.normalize('NFD', tweet).\
#                     encode('ascii', 'ignore').decode('utf-8'))
# df['main'] = df['main'].str.replace(r'[^\w\s]', '')
# df['main'].head()

Why we should use the tweets tokenizer:

In [ ]:
tknzr = TweetTokenizer()
tknzr.tokenize('mir gahts so so lala lol!! merd:X :D')

In [ ]:
'mir gahts so so lala lol!! :D'.split()

In [ ]:
tokenized = df['main'].map(lambda x: tknzr.tokenize(x))

In [ ]:
#tokenized.to_pickle('tokenized_strings.pkl')
#save time and read from this pkl

In [ ]:
# df['split'] = df['main'].str.split()
df['tokenized'] = tokenized

In [ ]:
def remove_stops(language, cleaned, frame):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    lang_set = stopwords.words(language)
    cleaned.loc[cleaned['lang'] == lang, frame] = cleaned.loc[cleaned['lang'] == lang, frame].\
        apply(lambda tweet: [word for word in tweet if word not in lang_set])

In [ ]:
remove_stops('english', df, 'tokenized')
remove_stops('french', df, 'tokenized')
remove_stops('german', df, 'tokenized')

In [ ]:
df.head()

In [ ]:
def stem_words(language, cleaned, frame):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    stemmer = SnowballStemmer(language)
        
    cleaned.loc[cleaned['lang'] == lang, frame] = cleaned.loc[cleaned['lang'] == lang, frame].\
        apply(lambda tweet: [stemmer.stem(word) for word in tweet])

In [ ]:
stem_words('english', df, 'tokenized')
stem_words('french', df, 'tokenized')
stem_words('german', df, 'tokenized')

In [ ]:
df.tokenized.to_pickle('tokenized_stemmed.pkl')

In [ ]:
DICT_PATH = "dictionary.csv"
dictionaries = pd.read_csv(DICT_PATH)
dictionaries.head()

In [ ]:
en_dict = dictionaries['english'].dropna()
fr_dict = dictionaries['french'].dropna()
de_dict = pd.concat([dictionaries['german'].dropna(), dictionaries['swiss_german'].dropna()])

In [ ]:
def dict_cleaning(lang):
    lang_dict = eval(lang + '_dict')
    lang_dict = lang_dict.astype(str).str.lower().\
                        apply(lambda expression: unicodedata.normalize('NFD', expression).\
                        encode('ascii', 'ignore').decode('utf-8'))

In [ ]:
dict_cleaning('en')
dict_cleaning('fr')
dict_cleaning('de')

In [ ]:
#Tokenizing
en_dict = en_dict.str.split()
fr_dict = fr_dict.str.split()
de_dict = de_dict.str.split()

In [ ]:
def dict_remove_stops(language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    lang_dict = eval(lang + '_dict')
    lang_set = stopwords.words(language)
        
    lang_dict = lang_dict.apply(lambda expression: [word for word in expression if word not in lang_set])

In [ ]:
#Removing stop words
dict_remove_stops('english')
dict_remove_stops('french')
dict_remove_stops('german')

In [ ]:
def dict_stem_words(language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    lang_dict = eval(lang + '_dict')
    stemmer = SnowballStemmer(language)
        
    lang_dict = lang_dict.apply(lambda expression: [stemmer.stem(word) for word in expression])

In [ ]:
#Stemming the words
dict_stem_words('english')
dict_stem_words('french')
dict_stem_words('german')

In [ ]:
df['merged'] = df.tokenized.map(lambda x:  ' '.join(x))
de_dict = de_dict.map(lambda x:  ' '.join(x))
en_dict = en_dict.map(lambda x:  ' '.join(x))
fr_dict = fr_dict.map(lambda x:  ' '.join(x))

In [ ]:
def getMatching(tweet_list, )

In [ ]:
df['tokenized'].head()

In [ ]:
def check_dict(tweet, dict_):
    """checks if dict entry matches tweet"""
    match = [ w for w in dict_ if w in tweet] #find matching for each entry
    return len(match) > 0 #at least one match

In [ ]:
english_filtered = df[df.lang == 'en'][df[df.lang == 'en']['merged'].map(lambda x: check_dict(x, en_dict))]

In [ ]:
english_filtered.main.sample(5)

In [ ]:
english_filtered.count()

In [ ]:
french_filtered = df[df.lang == 'fr'][df[df.lang == 'fr']['tokenized'].map(lambda x: check_dict(x, fr_dict))]
french_filtered.head()

In [ ]:
len(french_filtered)

In [ ]:
german_filtered = df[df.lang == 'de'][df[df.lang == 'de']['tokenized'].map(lambda x: check_dict(x, de_dict))]
german_filtered.head()

In [ ]:
len(german_filtered)

In [ ]:
#english_filtered.to_csv('english.csv')

In [ ]:
#english_filtered.sample(n=1000).to_csv('english_1000.csv')

In [ ]:
#french_filtered.sample(n=1000).to_csv('french_1000.csv')

In [ ]:
#move this above
import re
s = "string. With. Punctuation? ! àéè. : ; "
s = re.sub(r'[^\w\s]','',s)
s

In [ ]:
english_filtered_2 = english_filtered.tokenized.map(lambda x: [re.sub(r'[^\w\s]','',s) for s in x])

## LDA starts here

In [ ]:
dictionary = corpora.Dictionary(english_filtered_2)

In [ ]:
dictionary.filter_extremes(no_below=3, no_above=.5)

In [ ]:
print(dictionary)

In [ ]:
texts = english_filtered_2.tolist()

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
ldamodel = models.LdaMulticore(corpus, id2word=dictionary, num_topics=3, workers=3, iterations=100, passes=5) #takes like 5minutes on leo's pc

In [ ]:
ldamodel.show_topics()

In [ ]:
#ldamodel =  models.LdaModel.load('lda.model') #retrive lda model

In [ ]:
sent_to_cluster = list()
for n,doc in enumerate(corpus):
    if doc:
        cluster = max(ldamodel[doc],key=lambda x:x[1])
        sent_to_cluster.append(cluster[0])

In [ ]:
#ldamodel.save('lda.model') #save lda model

In [ ]:
vis_data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

In [ ]:
pyLDAvis.display(vis_data)

## Same for french etc:

In [ ]:
french_filtered_2 = french_filtered.tokenized.map(lambda x: [re.sub(r'[^\w\s]','',s) for s in x])

In [ ]:
french_filtered_2.head()

In [ ]:
fr_dictionary = corpora.Dictionary(french_filtered_2)
fr_dictionary.filter_extremes(no_below=5, no_above=.3)
fr_texts = french_filtered_2.tolist()
fr_corpus = [fr_dictionary.doc2bow(text) for text in fr_texts]

ldamodel_2 = models.LdaMulticore(fr_corpus, id2word=fr_dictionary, num_topics=10, workers=3, iterations=100, passes=5)

vis_data = pyLDAvis.gensim.prepare(ldamodel_2, fr_corpus, fr_dictionary)

pyLDAvis.display(vis_data)

## Same for german:

In [ ]:
german_filtered_2 = german_filtered.tokenized.map(lambda x: [re.sub(r'[^\w\s]','',s) for s in x])

In [ ]:
ger_dictionary = corpora.Dictionary(german_filtered_2)
ger_dictionary.filter_extremes(no_below=3, no_above=.5)
ger_texts = german_filtered_2.tolist()
ger_corpus = [ger_dictionary.doc2bow(text) for text in ger_texts]

ldamodel_2 = models.LdaMulticore(ger_corpus, id2word=ger_dictionary, num_topics=5, workers=3, iterations=100, passes=5)

vis_data = pyLDAvis.gensim.prepare(ldamodel_2, ger_corpus, ger_dictionary)

pyLDAvis.display(vis_data)